In [52]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [72]:
batchsize=8

In [93]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:/Users/user/New folder/data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:/Users/user/New folder/data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 1975 images belonging to 4 classes.
Found 492 images belonging to 4 classes.


In [94]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:/Users/user/New folder/data/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 433 images belonging to 4 classes.


In [95]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(4,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [96]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_583 (Conv2D)            (None, 39, 39, 32)   864         ['input_9[0][0]']                
                                                                                                  
 batch_normalization_574 (Batch  (None, 39, 39, 32)  96          ['conv2d_583[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_574 (Activation)    (None, 39, 39, 32)   0           ['batch_normalization_574[0

In [97]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [98]:
checkpoint = ModelCheckpoint(r'C:/Users/user/New folder/models/model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [99]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [103]:
model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=15)

Epoch 1/15
246/246 [==============================] - ETA: 0s - loss: 0.4001 - accuracy: 0.7946
Epoch 1: val_loss did not improve from 0.38147
246/246 [==============================] - 42s 171ms/step - loss: 0.4001 - accuracy: 0.7946 - val_loss: 0.4031 - val_accuracy: 0.7439 - lr: 0.0010
Epoch 2/15
246/246 [==============================] - ETA: 0s - loss: 0.3866 - accuracy: 0.7905
Epoch 2: val_loss improved from 0.38147 to 0.36946, saving model to C:/Users/user/New folder/models\model.h5
246/246 [==============================] - 45s 182ms/step - loss: 0.3866 - accuracy: 0.7905 - val_loss: 0.3695 - val_accuracy: 0.7684 - lr: 0.0010
Epoch 3/15
246/246 [==============================] - ETA: 0s - loss: 0.3728 - accuracy: 0.7936
Epoch 3: val_loss did not improve from 0.36946
246/246 [==============================] - 44s 179ms/step - loss: 0.3728 - accuracy: 0.7936 - val_loss: 0.4102 - val_accuracy: 0.7623 - lr: 0.0010
Epoch 4/15
246/246 [==============================] - ETA: 0s - loss

In [104]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

247/247 [==============================] - 33s 132ms/step - loss: 0.3240 - accuracy: 0.8415
0.32401061058044434
0.8415189981460571


In [105]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

62/62 [==============================] - 9s 149ms/step - loss: 0.3749 - accuracy: 0.7805
0.3749113380908966
0.7804877758026123


In [106]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)

55/55 [==============================] - 7s 127ms/step - loss: 0.3240 - accuracy: 0.8037
0.32401061058044434
0.8415189981460571
